Vou fazer esssa bosta aqui

Quero ver se não funciona!

In [3]:
import pandas as pd
from pathlib import Path

## Classe Vazoes

Yanase

In [4]:
# -*- coding: utf-8 -*-
"""Arquivo vazoes.txt."""
from pathlib import Path
from typing import Optional, Union

import pandas as pd
import numpy as np


def ler_arquivo_vazoes_txt(arquivo: Union[str, Path]) -> pd.DataFrame:
    """
    Leitura de um arquivo txt de vazões como dataframe.

    O dataframe é semelhante ao formato original de linhas e colunas.
    Como índice ficam o posto e ano, e como colunas os meses.

    Parameters
    ----------
    posto : int
        Número do posto com o qual foi feita a correlação.

    Returns
    -------
    DataFrame
        Dataframe com os dados lidos.

    """
    # Leitura do arquivo como dataframe, usando espaços como separador.
    df_arq = pd.read_csv(arquivo, header=None, delim_whitespace=True)
    # Nomeação das colunas e índices
    df_arq.columns = ['posto', 'ano', *range(1, 13)]
    df_arq.set_index(['posto', 'ano'], inplace=True)
    df_arq.columns.name = 'mes'

    df_arq = df_arq.astype(int)

    return df_arq


def _periodizar_df_arq(df_arq: pd.DataFrame) -> pd.DataFrame:
    """
    Cria um dataframe 'periodizado' (ano/mes x postos).

    Com o tempo como linhas e os postos como colunas, tem-se um dataframe
    mais simples com apenas duas dimensões, facilitando sua manipulação.

    Parameters
    ----------
    df_arq : DataFrame
        Dataframe com os dados lidos, oriundo de ler_arquivo_vazoes_txt.

    Returns
    -------
    DataFrame
        Dataframe com os dados 'periodizados'.

    """
    # Empilha os meses e desempilha o posto
    df_period = df_arq.stack().unstack('posto')

    # Transforma os indexes ano e mes em um index só que representa ambas
    # as informações de uma melhor forma. O nome do index continua sendo 'mes'
    df_period.index = pd.PeriodIndex(df_period.index.get_level_values('ano').astype(str)
                                   + '/'
                                   + df_period.index.get_level_values('mes').astype(str),
                                   freq='M')
    df_period.index.name = 'mes'

    # Se todas as colunas são 0, muda para NaN
    # Presume-se aqui que isso só acontece nos períodos finais, caso
    # contrário melhorar este trecho.
    df_period.loc[(df_period == 0).all(axis='columns')] = np.NaN
    # exclui as colunas com NaN
    df_period.dropna(inplace=True)
    # Sem NaN, é possível deixar as colunas com o tipo inteiro
    df_period = df_period.astype(int)

    return df_period


def _desperiodizar_df_arq(df_period: pd.DataFrame) -> pd.DataFrame:
    """
    Cria um dataframe 'desperiodizado', voltando ao formato tradicional.

    Parameters
    ----------
    df_period : DataFrame
        Dataframe com os dados 'periodizados'.

    Returns
    -------
    DataFrame
        Dataframe com os dados 'desperiodizados'.

    """
    df_arq = df_period.set_index([df_period.index.year,
                                df_period.index.month])
    df_arq.index.names = ['ano', 'mes']
    df_arq = df_arq.stack().unstack('mes').swaplevel()

    # Substitui os NaN por 0 e deixa como inteiro
    df_arq.fillna(0, inplace=True)
    df_arq = df_arq.astype(int)

    df_arq.sort_index(inplace=True)

    return df_arq


class VazoesTxt:
    """
    Classe que representa o arquivo vazoes.txt.

    Attributes
    ----------
    df_arquivo : DataFrame
        Representação mais fiel ao estilo do arquivo.
    df_periodizado : DataFrame
        Representação (ano/mes x posto). Simplifica cálculos e manipulações
        por ter apenas duas dimensões.

    """

    def __init__(self, arquivo: Optional[Union[str, Path]] = None):
        """
        Construtor do objeto.

        Parameters
        ----------
        arquivo : str or Path, optional
            Nome ou caminho do arquivo de vazões.txt utilizado.

        """
        self._filepath = Path(arquivo) if arquivo else None

        # Construção dos dataframes.
        # Poderiam ser tanto métodos como funções à parte,
        # discutir o que seria melhor
        self.df_arquivo = ler_arquivo_vazoes_txt(self._filepath) if arquivo else None
        self.df_period = _periodizar_df_arq(self.df_arquivo) if arquivo else None


    @classmethod
    def from_df_period(cls, df_period: pd.DataFrame):
        """
        Construtor a partir de um dataframe 'periodizado'.

        Parameters
        ----------
        df_period : DataFrame
            Dataframe com os dados 'periodizados' (PeriodIndex freq 'M').

        """
        self = cls()
        self.df_period = df_period
        self.df_arquivo = _desperiodizar_df_arq(self.df_period)

        return self

    def add_novo_periodo(self,
                         df_new_months: pd.DataFrame,
                         inplace: bool = False,
                         ) -> None:
        """
        Adiciona informações dos próximos meses.

        Parameters
        ----------
        df_new_months : DataFrame
            Dataframe com as informações a serem adicionadas.

        Returns
        -------
        if not inplace:
            VazoesTxt
                Um novo objeto com as informações concatenadas
        if inplace:
            None
                O atributos do objeto são modificados localmente.

        """
        # Verifica o próximo mês a ser preenchido
        next_month = self.df_period.index[-1] + pd.offsets.MonthEnd()
        # Calcula a diferença entre o próximo mês e o primeiro do período informado
        diff_months = next_month - df_new_months.index[0]
        # Ajusta o index
        df_new_months_ajust = df_new_months.copy()
        df_new_months_ajust.index = df_new_months.index + diff_months

        # Concatena com os dados originais
        df_new = pd.concat([self.df_period, df_new_months_ajust],
                            verify_integrity=True)

        # Se não muda localmente, retorna um novo objeto
        if inplace is False:
            return self.from_df_period(df_new)

        # Muda o objeto no local
        self.df_period = df_new
        # Desnormaliza
        self.df_arquivo = _desperiodizar_df_arq(self.df_period)

        return None


    def salvar_txt(self,
                   arquivo_destino: Union[str, Path],
                   ) -> None:
        """
        Salva o dataframe modificado em um novo arquivo de vazões.txt.

        Parameters
        ----------
        arquivo_destino : str or Path
            Nome ou caminho do arquivo de destino.

        """
        lista_linhas = list()
        for idx, row in self.df_arquivo.iterrows():
            lista_linhas.append(f"{idx[0]:3} {idx[1]:4}"
                                f"{row[1]:6}{row[2]:6}{row[3]:6}"
                                f"{row[4]:6}{row[5]:6}{row[6]:6}"
                                f"{row[7]:6}{row[8]:6}{row[9]:6}"
                                f"{row[10]:6}{row[11]:6}{row[12]:6}")

        conteudo_arquivo = '\n'.join(lista_linhas)

        with open(arquivo_destino, 'w') as file:
            file.write(conteudo_arquivo)
            file.write('\n')

## Cálculo da Correlação

def calc_corr_last12(df_hist: pd.DataFrame) -> pd.DataFrame:
    """
    Cálculo da correlação dos últimos 12 meses informados com anos anteriores.

    Parameters
    ----------
    df_hist : DataFrame
        Dataframe do histórico a ser analisado.
        As 12 últimas linhas devem ser os 12 meses com os quais será calculada
        a correlação com dados de anos anteriores.

    Returns
    -------
    DataFrame
        Dataframe com as correlações calculadas.
        Como index é informado o mês final do período de 12 meses com o qual
        foi calculada a correlação.

    """
    # pylint: disable=protected-access
    # Conferindo se o index está no formato desejado (pd.PeriodIndex freq=M)
    if not isinstance(df_hist.index.freq, pd._libs.tslibs.offsets.MonthEnd):
        raise Exception("Favor informar dataframe periodizado mensalmente.")

    df_ref = df_hist.copy()
    # Trecho de 12 meses com o qual será calculada a correlação
    df_trecho = df_ref.iloc[-12:]
    # Espalha o trecho por uma cópia do dataframe respeitando os meses
    df_other = df_ref.copy()
    for month in range(1, 13):
        df_other[df_other.index.month == month] = df_trecho[df_trecho.index.month == month].iloc[0]

    # Calcula a correlação
    df_corr = df_ref.rolling(12).corr(df_other)
    # Filtra os dados da correlação com o mês final
    df_corr = df_corr[df_corr.index.month == df_trecho.index.month[-1]]
    df_corr.index.name = 'mes_final'

    return df_corr

## Modelo

In [5]:
import pandas as pd
from typing import Optional

class ModeloCamila:
    """
    Classe que representa o um modelo de correlações simples e Teste da amplitude.
    """

    def __init__(self,
                 coluna: int,
                 posicao: int = 1,
                 ):
        """
        Criação do modelo.

        """
        self.coluna = coluna
        self.posicao = posicao

        # Atributos que serão definidos apenas ao realizar o fit
        self.df_base = None
        self.df_correlacao = None
        self._period_ = None
        self.correlacao_ = None
        self.correlacao_outros_ = None
        self.mes_final_periodo_ = None


    def fit(self, df_base: pd.DataFrame) -> 'ModeloCamila':
        """
        Cálculo das correlações.
        """
        self.df_base = df_base.copy()
        self.df_correlacao = calc_corr_last12(df_base)

        top_corr = self.df_correlacao[self.coluna].sort_values(ascending=False)

        # Dados que serão usados para previsão
        self._period_ = top_corr.index[self.posicao]
        #======================================================================
        mes_escolhido_ini = self._period_ + 1        
        
        #série com a primeiro mês da previsão extendida
        series_proximo_mes = self.df_base.loc[mes_escolhido_ini]
        #Renomeando index
        series_proximo_mes.name = self.df_base.index[-1] + 1
        #print(series_proximo_mes.to_frame().T)
        
        #Acrescentando o primeiro mês da previsão extendida nos dados originais
        df_base_modif = pd.concat([self.df_base, series_proximo_mes.to_frame().T])
        #print(df_base_modif)
        print(self._period_)
        print(mes_escolhido_ini)
        
        #Cálculo da razão
        previsao_refinitiv = self.df_base.iloc[-1]
        previsao_extendida = df_base_modif.iloc[-1]
        df_razao = previsao_refinitiv / previsao_extendida
        
        #Usando a função shift
        df_teste = df_base_modif.shift(periods=0, freq='1M')
        
        print(df_teste)
        
        #======================================================================
        # Outras informações sobre o modelo 'treinado'
        # Usando sufixo _ semelhante ao scikit-learn
        self.correlacao_ = top_corr.iloc[self.posicao]
        self.correlacao_outros_ = self.df_correlacao.loc[top_corr.index[self.posicao]]
        self.mes_final_periodo_ = self._period_.strftime('%Y-%m')

        # Retorna o próprio objeto para o caso de ser encadeado com .predict()
        return self


    def predict(self,
                num_meses: Optional[int] = None,
                ) -> pd.DataFrame:
        """
        Previsão para os próximos 'num_meses'.

        """
        # Mês seguinte ao período selecionado pelo modelo
        try:
            # Não havendo erro supomos que o fit foi realizado
            mes_escolhido_ini = self._period_ + 1
        except TypeError:
            # Podemos futuramente usar o erro sklearn.exceptions.NotFittedError,
            # mas para simplificar:
            raise Exception("Realizar o fit do modelo antes") from None

        # Primeiro mês a ser previsto
        next_month = self.df_base.index[-1] + 1

        # Se num_meses não for informado vai o final do ano
        num_meses = num_meses if num_meses else (13 - mes_escolhido_ini.month)

        # Dados do período escolhido
        df_escolhido = self.df_base.loc[mes_escolhido_ini:].iloc[:num_meses]

        # Ajusta o index para o novo período
        df_trecho_ajust = df_escolhido.copy()
        df_trecho_ajust.index = pd.period_range(next_month,
                                                periods=len(df_trecho_ajust),
                                                freq='M')

        return df_trecho_ajust


    def teste_amplitude(self):
        """Verifica se a razão entre os meses de previsão
        estão em um intervalo entre o razão mínina e a máxima do histórico.
        
        Caso não no intervalo 
        """
        #-------------
        #Teste da Amplitude
        
        #Cria uma razão com o último ano de previsão da Refinitiv pelo primeiro mês da previsão extendida
        # df_trecho = self.predict()
        
        df_trecho = self.predict()
        #Desperiodizar o index para fazer o cálculo da razão
       
        #dict_amplitude = {}
        df_original = pd.DataFrame(self.df_base)
        df_razao = df_trecho.describe()
        #df_razao = df_original.index[-1].values
        
        # #Resultado da razão em todos os postos, matriz de anos por posto
        # self.df_amplitude = df_razao
        # self.df_amplitude = df_razao.unstack('posto')
        
        # #Seleciona a amplitude do último ano
        # self.df_amplitude_previsao = self.df_amplitude.iloc[-1:].squeeze()
        # self.df_amplitude_previsao = self.df_amplitude_previsao.T        
        
        # #Drop na linha com as previsões para comparar com as demais
        # self.df_amplitude = self.df_amplitude.drop(self.df_amplitude.index[-1])
        
        # #Seleciona a maior e a menor razão do histórico
        # self.df_amplitude_maximo = self.df_amplitude.max(skipna=False)
        # self.df_amplitude_minima = self.df_amplitude.min(skipna=False)
        
        # #Verifica se a razão das previsões está no intervalo entre a máxima e a mínima do histórico
        # self.verifica_max = self.df_amplitude_previsao >= self.df_amplitude_maximo 
        # self.verifica_min = self.df_amplitude_previsao <= self.df_amplitude_minima
        
        
        # if verifica_max.iloc[self.coluna] is True:
        #     self.posicao += 1
        # if verifica_min.iloc[self.coluna] is True:
        #     self.posicao += 1
        return df_original


## Teste

In [6]:

arquivo = Path('C:/Workspace/cenarios_por_correlacao_de_vazoes/teste_arquivos_entrada/VAZOES-P50.txt')

# Criação do objeto de vazões
vazoes = VazoesTxt(arquivo)

model = ModeloCamila(coluna=6, posicao=1)
# Parametrização do modelo
model.fit(vazoes.df_period)
# Predição para um novo período
novo_trecho = model.predict()

# Junção do período do arquivo com o período previsto
vazoes_new = vazoes.add_novo_periodo(novo_trecho)
# Salva novo arquivo de vazões no formato txt

# Apenas para visualização das correlações do principais postos
#correlacoes_principais = model.df_correlacao[POSTOS_PRINCIPAIS.keys()]
#correlacoes_principais.columns = POSTOS_PRINCIPAIS.values()

#----------------------------------------------------------

#a1 = a1.loc[0]


1967-04
1967-05
posto    1    2     6     7     8     9     10    11    12   14   ...  304  \
1931-01  178  178  1476  1690  1737  1759  1819  1939  2210  152  ...    0   
1931-02  371  371  2964  3318  3385  3408  3483  3661  4115  220  ...    5   
1931-03  326  326  2167  2471  2532  2558  2632  2788  3158  143  ...    0   
1931-04  479  479  1585  1827  1879  1903  1969  2100  2393  107  ...    0   
1931-05  332  332  1254  1428  1463  1479  1522  1614  1828   60  ...    0   
...      ...  ...   ...   ...   ...   ...   ...   ...   ...  ...  ...  ...   
2023-01  253  253  1611  1797  1829  1846  1899  2011  2241   76  ...    0   
2023-02  236  236  1541  1727  1776  1791  1850  1965  2226   75  ...    0   
2023-03  197  197  1304  1482  1527  1546  1598  1704  1948   65  ...    0   
2023-04   93   93   574   661   684   694   722   775   902   34  ...    0   
2023-05  108  108   880  1003  1028  1039  1071  1136  1288   45  ...    0   

posto    306  313   314  315  316  317  318  31

In [7]:
a1 = model.predict()
a1

posto,1,2,6,7,8,9,10,11,12,14,...,304,306,313,314,315,316,317,318,319,320
2023-05,108,108,880,1003,1028,1039,1071,1136,1288,45,...,0,188,171,587,171,171,0,24,22,0
2023-06,112,112,763,869,890,900,926,981,1112,42,...,0,183,148,462,166,166,0,36,33,7
2023-07,98,98,624,709,726,734,755,800,906,30,...,0,158,120,378,142,142,0,27,25,3
2023-08,69,69,546,621,636,643,661,700,794,26,...,0,122,104,295,113,113,0,17,16,0
2023-09,73,73,503,568,582,587,603,637,720,24,...,0,117,113,267,109,109,0,34,31,11
2023-10,81,81,523,572,580,582,591,617,690,25,...,0,131,149,272,122,122,0,31,28,10
2023-11,149,149,1080,1199,1218,1223,1242,1296,1447,44,...,0,180,166,519,167,167,0,56,51,20
2023-12,141,141,1303,1456,1485,1495,1528,1607,1808,61,...,0,189,258,751,172,172,0,46,42,14


In [10]:
a2 = model.teste_amplitude()
a2

posto,1,2,6,7,8,9,10,11,12,14,...,304,306,313,314,315,316,317,318,319,320
mes,,,,,,,,,,,,,,,,,,,,,
1931-01,178,178,1476,1690,1737,1759,1819,1939,2210,152,...,0,193,311,890,176,176,0,62,58,28
1931-02,371,371,2964,3318,3385,3408,3483,3661,4115,220,...,5,195,464,2025,195,190,10,82,76,35
1931-03,326,326,2167,2471,2532,2558,2632,2788,3158,143,...,0,195,538,1529,183,183,3,89,80,50
1931-04,479,479,1585,1827,1879,1903,1969,2100,2393,107,...,0,195,383,910,185,185,4,43,40,20
1931-05,332,332,1254,1428,1463,1479,1522,1614,1828,60,...,0,190,291,517,173,173,0,33,30,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12,184,184,1106,1192,1214,1223,1249,1306,1444,58,...,0,188,258,518,171,171,0,52,62,26
2023-01,253,253,1611,1797,1829,1846,1899,2011,2241,76,...,0,194,320,1108,177,177,2,69,98,34
2023-02,236,236,1541,1727,1776,1791,1850,1965,2226,75,...,0,191,358,1425,174,174,0,71,60,35


In [17]:
from matplotlib.pyplot import fill


a3 = a2.copy()
#Desloca os valores
a3 = a3.shift(1, fill_value=0.0)
a3

posto,1,2,6,7,8,9,10,11,12,14,...,304,306,313,314,315,316,317,318,319,320
mes,,,,,,,,,,,,,,,,,,,,,
1931-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1931-02,178,178,1476,1690,1737,1759,1819,1939,2210,152,...,0,193,311,890,176,176,0,62,58,28
1931-03,371,371,2964,3318,3385,3408,3483,3661,4115,220,...,5,195,464,2025,195,190,10,82,76,35
1931-04,326,326,2167,2471,2532,2558,2632,2788,3158,143,...,0,195,538,1529,183,183,3,89,80,50
1931-05,479,479,1585,1827,1879,1903,1969,2100,2393,107,...,0,195,383,910,185,185,4,43,40,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12,124,124,767,814,832,834,848,888,966,48,...,0,185,184,384,168,168,0,48,42,24
2023-01,184,184,1106,1192,1214,1223,1249,1306,1444,58,...,0,188,258,518,171,171,0,52,62,26
2023-02,253,253,1611,1797,1829,1846,1899,2011,2241,76,...,0,194,320,1108,177,177,2,69,98,34


In [130]:
#Guardar essa imformação
#Resultado da razão de todos os meses, anos e postos

#list comprehension 

#razao = a3/a2
#mask = razao.isin([np.inf, np.NaN]) == 0
# razao.isin([np.inf, np.nan])  == 00:
    
#razao_teste = razao[mask]
#razao_teste.fillna(0.0)

# Retorna a razão, substituindo os valores de NaN ou infinto igual a zero.
razao = a3.div(a2)
razao.replace([np.inf, np.nan], 0, inplace=True)
razao

posto,1,2,6,7,8,9,10,11,12,14,...,304,306,313,314,315,316,317,318,319,320
mes,,,,,,,,,,,,,,,,,,,,,
1931-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1931-02,0.479784,0.479784,0.497976,0.509343,0.513146,0.516138,0.522251,0.529637,0.537060,0.690909,...,0.0,0.989744,0.670259,0.439506,0.902564,0.926316,0.000000,0.756098,0.763158,0.800000
1931-03,1.138037,1.138037,1.367790,1.342776,1.336888,1.332291,1.323328,1.313128,1.303040,1.538462,...,0.0,1.000000,0.862454,1.324395,1.065574,1.038251,3.333333,0.921348,0.950000,0.700000
1931-04,0.680585,0.680585,1.367192,1.352490,1.347525,1.344193,1.336719,1.327619,1.319682,1.336449,...,0.0,1.000000,1.404700,1.680220,0.989189,0.989189,0.750000,2.069767,2.000000,2.500000
1931-05,1.442771,1.442771,1.263955,1.279412,1.284347,1.286680,1.293693,1.301115,1.309081,1.783333,...,0.0,1.026316,1.316151,1.760155,1.069364,1.069364,0.000000,1.303030,1.333333,1.176471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12,0.673913,0.673913,0.693490,0.682886,0.685338,0.681930,0.678943,0.679939,0.668975,0.827586,...,0.0,0.984043,0.713178,0.741313,0.982456,0.982456,0.000000,0.923077,0.677419,0.923077
2023-01,0.727273,0.727273,0.686530,0.663328,0.663751,0.662514,0.657715,0.649428,0.644355,0.763158,...,0.0,0.969072,0.806250,0.467509,0.966102,0.966102,0.000000,0.753623,0.632653,0.764706
2023-02,1.072034,1.072034,1.045425,1.040533,1.029842,1.030709,1.026486,1.023410,1.006739,1.013333,...,0.0,1.015707,0.893855,0.777544,1.017241,1.017241,0.000000,0.971831,1.633333,0.971429


In [100]:
#Seleciona o mês
from calendar import month


a6 = razao.index.copy()
#a6.asfreq('M', how='S')

In [150]:
#razao.nlargest(10, columns=[1])
#Exclui a última linha porque a mesma é usada para verificação do teste da amplitude
#E o primeiro valor porque não há razão do mesmo, porque não tem o mês anterior
#Exclui a previsão da Refinitiv para utilizar somente o histórico
a5 = razao.iloc[1: -7, :]
a5

posto,1,2,6,7,8,9,10,11,12,14,...,304,306,313,314,315,316,317,318,319,320
mes,,,,,,,,,,,,,,,,,,,,,
1931-02,0.479784,0.479784,0.497976,0.509343,0.513146,0.516138,0.522251,0.529637,0.537060,0.690909,...,0.0,0.989744,0.670259,0.439506,0.902564,0.926316,0.000000,0.756098,0.763158,0.800000
1931-03,1.138037,1.138037,1.367790,1.342776,1.336888,1.332291,1.323328,1.313128,1.303040,1.538462,...,0.0,1.000000,0.862454,1.324395,1.065574,1.038251,3.333333,0.921348,0.950000,0.700000
1931-04,0.680585,0.680585,1.367192,1.352490,1.347525,1.344193,1.336719,1.327619,1.319682,1.336449,...,0.0,1.000000,1.404700,1.680220,0.989189,0.989189,0.750000,2.069767,2.000000,2.500000
1931-05,1.442771,1.442771,1.263955,1.279412,1.284347,1.286680,1.293693,1.301115,1.309081,1.783333,...,0.0,1.026316,1.316151,1.760155,1.069364,1.069364,0.000000,1.303030,1.333333,1.176471
1931-06,1.469027,1.469027,1.597452,1.588432,1.585049,1.583512,1.578838,1.573099,1.567753,1.304348,...,0.0,1.027027,1.316742,1.389785,1.029762,1.029762,0.000000,1.269231,1.250000,1.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05,1.542857,1.542857,1.493243,1.486111,1.486486,1.488550,1.491682,1.486159,1.478770,1.518519,...,0.0,1.326531,1.428571,1.536585,1.592308,1.461538,0.000000,1.414634,1.432432,1.777778
2022-06,1.206897,1.206897,1.174603,1.177570,1.179954,1.180180,1.181223,1.184426,1.202465,1.173913,...,0.0,1.300885,1.272727,1.389831,1.354167,1.354167,0.000000,0.953488,0.948718,0.900000
2022-07,1.137255,1.137255,1.281356,1.300912,1.302671,1.302053,1.304843,1.311828,1.324009,1.210526,...,0.0,1.313953,1.193798,1.168317,1.315068,1.315068,0.000000,1.433333,1.444444,1.428571


In [154]:
#Funciona
#a5.groupby(pd.Grouper(freq='Y')).last()
#Também funciona
#a5.resample('Y').last()

a6[a6.month==4]


PeriodIndex(['1931-04', '1932-04', '1933-04', '1934-04', '1935-04', '1936-04',
             '1937-04', '1938-04', '1939-04', '1940-04', '1941-04', '1942-04',
             '1943-04', '1944-04', '1945-04', '1946-04', '1947-04', '1948-04',
             '1949-04', '1950-04', '1951-04', '1952-04', '1953-04', '1954-04',
             '1955-04', '1956-04', '1957-04', '1958-04', '1959-04', '1960-04',
             '1961-04', '1962-04', '1963-04', '1964-04', '1965-04', '1966-04',
             '1967-04', '1968-04', '1969-04', '1970-04', '1971-04', '1972-04',
             '1973-04', '1974-04', '1975-04', '1976-04', '1977-04', '1978-04',
             '1979-04', '1980-04', '1981-04', '1982-04', '1983-04', '1984-04',
             '1985-04', '1986-04', '1987-04', '1988-04', '1989-04', '1990-04',
             '1991-04', '1992-04', '1993-04', '1994-04', '1995-04', '1996-04',
             '1997-04', '1998-04', '1999-04', '2000-04', '2001-04', '2002-04',
             '2003-04', '2004-04', '2005-04', '2006-

In [118]:
valor_maximo = a5.max()
valor_maximo

posto
1       2.972222
2       2.972222
6       2.722171
7       2.579075
8       2.546099
         ...    
316     2.116883
317    39.000000
318     4.052632
319     4.117647
320     9.000000
Length: 217, dtype: float64

In [119]:
valor_minimo = a5.min()
valor_minimo

posto
1      0.247012
2      0.247012
6      0.201765
7      0.219689
8      0.224776
         ...   
316    0.442105
317    0.000000
318    0.169014
319    0.187500
320    0.000000
Length: 217, dtype: float64

In [67]:
#Seleciona o razão da previsão 
#último meês de previsão da Refinitiv e primeiro mês da previsão extendida
razao_previsao = razao.iloc[-1, :]
razao_previsao

posto
1      2.118280
2      2.118280
6      2.271777
7      2.242057
8      2.232456
         ...   
316    1.398374
317    0.000000
318    2.000000
319    1.509434
320    2.071429
Name: 2023-04, Length: 217, dtype: float64

In [77]:

mask = razao_previsao[razao_previsao >= valor_maximo]
mask

posto
34     2.015540
45     1.962835
154    1.521739
244    1.996711
245    1.963224
296    1.380000
Name: 2023-04, dtype: float64